In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width: 100% !important; }</style>"))

In [ ]:
from datetime import datetime
import os

solutions_list = []
max_runs = 10
dct_with_header = False
save_results = True
_instance = 'CMT1'
_cluster_type = 'K-MEANS'

# Creación del archivo de texto plano con los resultados
now = datetime.now()
dt_string = now.strftime('%Y%m%d_%H-%M-%S')
aco_cluster = 'Restricted Ant + ' + _cluster_type # Titulo del archivo .txt
ant_type = 'RA' # FA o RA (FREE ANT - RESTRICTED ANT) para el titulo del archivo
cluster_type = _cluster_type.replace('-', '').upper()[0:4] # KMEN o KMED (K-MEANS - K-MEDOIDS) para el titulo del archivo
instance = _instance # Nombre de la carpeta
file_folder = 'instances_results/' + instance # Carpeta raiz de resultados
file_folder_img = 'instances_results/' + instance + '/img_results'
if not os.path.exists(file_folder): os.makedirs(file_folder)
if not os.path.exists(file_folder_img): os.makedirs(file_folder_img)
file_name = ant_type + '_' + cluster_type + '_' + dt_string + '.txt'

for run in range(max_runs): 
    %matplotlib inline
    from restricted_bwacs_model import RestrictedBWACS
    aco_model = RestrictedBWACS(
        # Nombre de la instancia
        instance = 'CVRPLIB/CMT/' + _instance,
        # Número máximo de nodos
        max_nodes = 999, 
        # Costo a minimizar
        ant_type = 'emvrp',
        # Tipo de clustering | kmedoids - kmeans |
        cluster_type = _cluster_type.replace('-', '').lower(),
        # Métrica de distancia | euclidian - manhattan |
        metric = 'euclidian',
        # Peso del vehículo | corresponde al producto entre la cápacidad máxima y un porcentaje = Q * 0.15
        tare = 0.15,
        # Número máximo de iteraciones
        max_iterations = 50,
        # Cántidad de hormigas por iteracion
        max_ants = 25,
        # Inicializar cada hormiga en un nodo disinto dentro de una lista de candidatos. De lo contrario cada hormiga comenzara su recorrido en el depot | 0 - 1 |
        start_ant_on_best_nodes = 1,
        # Valor de importancia de las feromonas / Mejor alpha: [1, 3]
        alpha = 3,
        # Valor de importancia de la distancia
        beta = 2,
        # Valor de importancia de la energia
        gamma = 3,
        # Valor de importancia del ahorro de distancia
        delta = 1,
        # Valor de importancia del ahorro de energia
        eta = 1,
        # Valor de importancia de la utilización de capacidad
        mi = 0,
        # Que tipo de estrategia de actualización de feromonas usar (1 / BestQuality) o Ant Weight Strategy | 0 - 1 |
        pheromone_updating_strategy = 1,
        # Actualización de feromonas por hormiga | 0 - 1 |
        local_ant_update_pheromones = 0,
        # Actualización de feromonas por la mejor de la iteración | 0 - 1 |
        best_iteration_update_pheromones = 1,
        # Actualización de feromonas por la mejor global | 0 - 1 |
        best_global_update_pheromones = 1,
        # Penalizar peor solución de la iteración | 0 - 1 |
        penalize_worst_solution = 1,
        # Mutar matriz de feromonas | 0 - 1 |
        mutate_pheromones_matrix = 1,
        # Ratio de evaporación de las feromonas | (1 - p) |
        p = 0.02,
        # Probabilidad de mutación de la matriz de feromonas
        Pm = 0.3, 
        # Intensidad de la mutación de la matriz de feromonas
        sigma = 2, 
        # Probabilidad de elegir el mejor nodo
        q0 = 0.3, 
        # Constante para calcular las funciones de costos entre otras | Q / BestTour |
        Q = 1,
        # El tipo de información heuristica a utilizar
        # 0 - Solo la información de distancias / Mejor beta: 5
        # 1 - Solo la información de energias / Mejor gamma: 5
        # 2 - Información de distancias X energias / Mejores parametros: 5 - 5
        # 3 - Solo información de ahorro de distancia | saving's algorithm | / Mejor delta: 10
        # 4 - Solo información de ahorro de energia | saving's algorithm | / Mejor eta: 10
        # 5 - Información de distancias X ahorro de distancia / Mejores parametros: 4 - 5
        # 6 - Información de energias X ahorro de energia / Mejores parametros: 5 - 5
        # 7 - Información de distancias X información energias X ahorro de distancia / Mejores parametros: 3 - 4 - 3
        # 8 - Información de distancias X información energias X ahorro de energias / Mejores parametros: 3 - 4 - 3
        # 9 - Información de distancias X información de energias X ahorro de distancia X información de capacidad / Mejores parametros: 2 - 4 - 4 - 2
        # 10 - Información de distancias X información de energias X ahorro de energia X información de capacidad / Mejores parametros: 2 - 4 - 4 - 2
        # 11 - Información de distancias X información de energias X ahorro de distancia X ahorro de energia X información de capacidad / Parametros: 2 - 4 - 3 - 4 - 1
        heuristic_type = 8,
        # Busqueda local a la solución individual de cada hormiga | 0 - 1 |
        ls_ant_solution = 0,
        # Busqueda local a la solución final | 0 - 1 |
        ls_final_solution = 0,
        # Output folder/name solution draw
        output_sol_img = file_folder_img + '/' + str(run + 1) + '__' + ant_type + '_' + cluster_type + '_' + dt_string + '.png'
    )
    
    solution_energy, solution_distance, solution_time = aco_model.solve()
    solutions_list.append((solution_energy, solution_distance, solution_time))

print(solutions_list)
print("Best Solution: " + str(min(solutions_list, key=lambda x: x[0])) + " on run: " + str(solutions_list.index(min(solutions_list, key=lambda x: x[0])) + 1))

if (save_results):
    file = open(file_folder + '/' + file_name, 'a')
    if dct_with_header: file.write('Resultados para la ' + instance + ' (' + aco_cluster + ') [energia, distancia, tiempo de ejecución]\n\n')
    for i, solution in enumerate(solutions_list):
        if dct_with_header: file.write(str(i + 1) + '| ' + str(solution) + '\n')
        else: 
            str_sol = str(solution).replace('(', '').replace(')', '').replace(',', '').replace('.', ',')
            file.write(str_sol + '\n')

    file.close()